In [3]:
from QUBO_grover import *

from docplex.mp.model import Model
from qiskit.primitives import StatevectorSampler
from qiskit_optimization.algorithms import GroverOptimizer, MinimumEigenOptimizer
from qiskit_optimization.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_optimization.translators import from_docplex_mp

In [4]:
from docplex.mp.model import Model
import numpy as np

# Example QUBO
n = 3
Q = np.array([[1, 2, -5],
              [0, 3, 0],
              [0, 0, 4]])

# Symmetrize Q (QUBO should be symmetric)
Q = Q + Q.T - np.diag(Q.diagonal())

# Create binary variables
model = Model()
x = [model.binary_var(name=f"x{i}") for i in range(n)]

# Build objective from QUBO
objective = model.sum(Q[i,j] * x[i] * x[j] for i in range(n) for j in range(i, n))
model.minimize(objective)

# Optional: solve
solution = model.solve()
print(solution)

solution for: docplex_model3
objective: 0
status: OPTIMAL_SOLUTION(2)



In [ ]:
model, qp = qubo_to_model(Q)
print(qp.prettyprint())


grover_optimizer = GroverOptimizer(3, num_iterations=10, sampler=StatevectorSampler(seed=123))
results = grover_optimizer.solve(qp)
print(results.prettyprint())

NameError: name 'qp' is not defined

In [7]:
print(results.prettyprint())

objective function value: 4.0
variable values: x0=1.0, x1=1.0, x2=0.0
status: SUCCESS


In [6]:
probs_array

[{'0100010': 0.1240234375,
  '0000000': 0.1083984375,
  '0010000': 0.1337890625,
  '0110010': 0.11328125,
  '1111110': 0.1298828125,
  '1101000': 0.1240234375,
  '1000010': 0.12890625,
  '1011000': 0.1376953125},
 {'1010110': 0.1240234375,
  '1000000': 0.1083984375,
  '0100000': 0.1337890625,
  '0001110': 0.11328125,
  '0011110': 0.1298828125,
  '0110000': 0.1240234375,
  '1100110': 0.12890625,
  '1111100': 0.1376953125},
 {'0010010': 0.1240234375,
  '1110000': 0.1083984375,
  '1000100': 0.1337890625,
  '1101010': 0.11328125,
  '1011010': 0.1298828125,
  '0100100': 0.1240234375,
  '0000010': 0.12890625,
  '0110100': 0.1376953125},
 {'1011010': 0.49609375, '1101010': 0.50390625},
 {'0101010': 0.1240234375,
  '1100000': 0.1083984375,
  '1010000': 0.1337890625,
  '0111010': 0.11328125,
  '1110110': 0.1298828125,
  '0001000': 0.1240234375,
  '1001010': 0.12890625,
  '0011000': 0.1376953125},
 {'1010110': 0.1240234375,
  '1000000': 0.1083984375,
  '0100000': 0.1337890625,
  '0001110': 0.113